---
# **Compute Nino3.4 index from CMIP5 SSTs**

In [1]:
import sys; sys.path.append('/rds/general/user/cb2714/home/01_wwa/'); from wwa import *

synda_path = "/rds/general/project/wwa/ephemeral/synda_clair/data/cmip5/output1/"

## **Download model SSTs**
_Using daily files rather than monthly, since these were available for more of the required models_

SSTs downloaded using synda - modify synda defaults file to contain
```
# Defaults for CMIP5 download

project=CMIP5
model=CM5A-LR CM5A-MR CNRM-CM5 CSIRO-Mk3-6-0 CanESM2 EC-EARTH GFDL-ESM2M GFDL-ESM2G HadGEM2-ES IPSL-CM5A-MR MIROC5 MPI-ESM-LR MPI-ESM-MR NorESM1-M 
experiment=rcp85 historical

latest=true
local_path_format=custom
timeslice=19000101-20501231
local_path_drs_template=CMIP5/%(model)s/%(ensemble)s/
```
Then
```
synda install variable=tos frequency=day ensemble=r1i1p1
synda install variable=tos frequency=day ensemble=r12i1p1 model=EC-EARTH 
```
CanESM2 has an extra historical file, which needs to be deleted before compiling  
IPSL-CM5A-MR_r1i1p1 downloaded manually (monthly res)  
EC-EARTH_r12i1p1 doesn't seem to be available at all for historical period

In [107]:
# list the models for which SSTs are required
models_needed = sorted(list(set(["_".join(fnm.split("_")[3:5]) for fnm in glob.glob("/rds/general/project/wwa/ephemeral/amazon_drought/cordex/spi/*.dat")])))

In [109]:
# check models for which daily TOS was available
fl_day = glob.glob(synda_path+"*/*/*/day/*/*/*/")
models_day = sorted(list(set([fp.split("/")[11]+"_"+fp.split("/")[-2] for fp in fl_day])))

# how many models lack SSTs?
sorted([m for m in models_needed if not m in models_day])

['EC-EARTH_r12i1p1', 'IPSL-CM5A-MR_r1i1p1']

# **Compute Nino3.4 and tropical SSTs**

##  **Extract SST time series**

In [144]:
# loop over raw files & save monthly SST series
for m in models_day:
    
    print(m)
    fl = glob.glob(synda_path+"*/"+m.split("_")[0]+"/*/day/*/*/"+m.split("_")[1]+"/*/*/*.nc")
    
    for fnm in fl:
        new_fnm = "cmip5_sst/nino34_"+"_".join(fnm.split("_")[3:])
        if os.path.exists(new_fnm): continue
            
        da = convert_units_to(wrap_lon(xr.open_dataset(fnm).tos.resample(time = "MS").mean()), "degC")
        
        if "latitude" in da.dims:
            da = da.rename(latitude = "lat", longitude = "lon")
            
        xydims = da.cf.axes["X"] + da.cf.axes["Y"]
        
        nino_box = np.logical_and(np.logical_and(da.lon >= -180, da.lon <= -120), np.logical_and(da.lat >= -5, da.lat <= 5))
        pm20_box = np.logical_and(da.lat >= -20, da.lat <= 20)
        nino34 = da.where(nino_box).mean(xydims)
        pm20 = da.where(pm20_box).mean(xydims)
        
        xr.merge([nino34.rename("nino34"), pm20.rename("pm20")]).to_netcdf(new_fnm)

CNRM-CM5_r1i1p1
CSIRO-Mk3-6-0_r1i1p1
CanESM2_r1i1p1
GFDL-ESM2G_r1i1p1
GFDL-ESM2M_r1i1p1
HadGEM2-ES_r1i1p1
MIROC5_r1i1p1
MPI-ESM-LR_r1i1p1
MPI-ESM-MR_r1i1p1
NorESM1-M_r1i1p1


In [140]:
# loop over manually uploaded files
for fnm in glob.glob("tmp/*.nc"):
    
    new_fnm = "cmip5_sst/nino34_"+"_".join(fnm.split("_")[3:])
    if os.path.exists(new_fnm): continue
    
    da = convert_units_to(wrap_lon(xr.open_dataset(fnm).tos), "degC")
    da = da.assign_coords(time = pd.date_range(start = fnm[-16:-12]+"-"+fnm[-12:-10]+"-01", end = fnm[-9:-5]+"-"+fnm[-5:-3]+"-01", freq = "MS"))

    if "latitude" in da.dims:
        da = da.rename(latitude = "lat", longitude = "lon")

    xydims = da.cf.axes["X"] + da.cf.axes["Y"]

    nino_box = np.logical_and(np.logical_and(da.lon >= -180, da.lon <= -120), np.logical_and(da.lat >= -5, da.lat <= 5))
    pm20_box = np.logical_and(da.lat >= -20, da.lat <= 20)
    nino34 = da.where(nino_box).mean(xydims)
    pm20 = da.where(pm20_box).mean(xydims)

    xr.merge([nino34.rename("nino34"), pm20.rename("pm20")]).to_netcdf(new_fnm)

## **Compile into Nino & detrended Nino time series**

In [2]:
fpath = "/rds/general/project/wwa/live/data/cmip5-nino34/"

In [69]:
fl = list(set([fnm.split("_")[2]+"_"+fnm.split("_")[4] for fnm in glob.glob("cmip5_sst/*.nc")]))

for fnm in fl:

    hist = convert_calendar(xr.open_mfdataset("cmip5_sst/*"+"_historical_".join(fnm.split("_"))+"*.nc"), "default", align_on = "date").load()
    rcp85 = convert_calendar(xr.open_mfdataset("cmip5_sst/*"+"_rcp85_".join(fnm.split("_"))+"*.nc"), "default", align_on = "date").load()

    if hist.time.max() >= rcp85.time.min():
        ds = xr.concat([hist.sel(time = slice(None, rcp85.time.min()-1)), rcp85], "time")
    else:
        ds = xr.concat([hist, rcp85], "time")

    # normalise Nino index to 1980-2010
    nino = ds.nino34.assign_attrs(note = "Mean of tos from 170W-120W, 5S-5N")
    nino = nino - nino.sel(time = slice("1980", "2010")).mean()
    
    # normalise tropical SSTs to 1980-2010
    pm20 = ds.pm20
    pm20 = pm20 - pm20.sel(time = slice("1980", "2010")).mean()
    
    # save Nino & detrended Nino as single dataset
    nino_det = (nino - pm20).rename("nino34_det").assign_attrs(note = "Nino3.4 detrended by subtracting mean of tos from 20S-20N")
    
    xr.merge([nino, nino_det]).to_netcdf(fpath+"nino34det_cmip5_"+fnm+".nc")

---
# **Compute Nino3.4 index from monthly CMIP6 SSTs**

In [4]:
import sys; sys.path.append('/rds/general/user/cb2714/home/01_wwa/'); from wwa import *

synda_path = "/rds/general/project/wwa/ephemeral/synda_clair/data/CMIP6/"

In [17]:
mdl_list = sorted(glob.glob(synda_path+"*"))

##  **Extract SST time series**

In [72]:
# loop over raw files & save monthly SST series
for m in mdl_list:
    
    print(m.split("/")[-1])
    fl = glob.glob(m+"/tos_*.nc")
    
    for fnm in fl:
        new_fnm = "cmip6_sst/nino34_"+"_".join(fnm.split("_")[3:])
        if os.path.exists(new_fnm): continue
            
        da = convert_calendar(convert_units_to(wrap_lon(xr.open_dataset(fnm).tos), "degC"), "default", align_on = "year")
        if "latitude" in da.dims: da = da.rename(latitude = "lat", longitude = "lon")
        
        if not "lat" in da.dims: continue
                    
        nino_box = np.logical_and(np.logical_and(da.lon >= -180, da.lon <= -120), np.logical_and(da.lat >= -5, da.lat <= 5))
        nino34 = da.where(nino_box).mean(["lat", "lon"])

        pm20_box = np.logical_and(da.lat >= -20, da.lat <= 20)
        pm20 = da.where(pm20_box).mean(["lat", "lon"])
        
        xr.merge([nino34.rename("nino34"), pm20.rename("pm20")]).to_netcdf(new_fnm)
        
clear_output(wait = False)
print("Done.")

Done.


## **Compile into Nino & detrended Nino time series**

In [90]:
fl = list(set([fnm.split("_")[2]+"_"+fnm.split("_")[4] for fnm in glob.glob("cmip6_sst/*.nc")]))

for fnm in fl:
    
    ds = xr.open_mfdataset("cmip6_sst/*"+fnm.split("_")[0]+"*.nc")
    
    # normalise Nino index to 1980-2010
    nino = ds.nino34.assign_attrs(note = "Mean of tos from 170W-120W, 5S-5N")
    nino = nino - nino.sel(time = slice("1980", "2010")).mean()
    
    # normalise tropical SSTs to 1980-2010
    pm20 = ds.pm20
    pm20 = pm20 - pm20.sel(time = slice("1980", "2010")).mean()
    
    # save Nino & detrended Nino as single dataset
    nino_det = (nino - pm20).rename("nino34_det").assign_attrs(note = "Nino3.4 detrended by subtracting mean of tos from 20S-20N")
    
    xr.merge([nino, nino_det]).to_netcdf("cmip6-nino34/nino34det_cmip6_"+fnm+".nc")

# CMIP6 

In [ ]:
! wget https://object-store.os-api.cci2.ecmwf.int/cci2-prod-cache-1/2025-03-04/4f22debdd9dc638a31b28cbd7addf341.zip
! wget https://object-store.os-api.cci2.ecmwf.int/cci2-prod-cache-1/2025-03-04/f8b3ff81c7527a7d955b65399411341e.zip
! unzip f8b3ff81c7527a7d955b65399411341e.zip

In [9]:
ds = xr.open_mfdataset("tos_Omon_*.nc").tos.load()

In [35]:
rm_tropics = np.logical_and(ds.latitude >= -20, ds.latitude <= 20)
rm_nino34 = np.logical_and(np.logical_and(ds.latitude >= -5, ds.latitude <= 5), np.logical_and(ds.longitude >= -180, ds.longitude <= -120))

In [36]:
ts_tropics = ds.where(rm_tropics == 1).mean(["i","j"])
ts_nino34 = ds.where(rm_nino34 == 1).mean(["i","j"])
ts_det = ts_nino34 - ts_tropics

In [52]:
ts_det = ts_det.convert_calendar("standard")
ts_det = ts_det.assign_coords(time = ts_det.time.dt.floor("D"))
ts_det.to_netcdf("/rds/general/user/cb2714/home/00_WWA_project_folder/live/data/cmip6-modes-of-variability_CVDP/nino34_cmip6/nino34_cmip6_FGOALS-G3_r1i1p1f1.nc")